In [27]:
import os
import gc
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Configuration

Using device: cuda


In [29]:
# Configuration
rows, cols = 200, 200
n_neurons = 100
n_feature = 9
batch_size = 70
num_classes = 10
out_fRules = torch.randn(n_feature, n_neurons).to(device)  # Placeholder for fuzzy rules


def gaussian(x, mu, sigma):
    #print(f"x shape: {x.shape}, mu shape: {mu.shape}, sigma shape: {sigma.shape}")  # Debugging

    # Ensure mu and sigma match x shape along the last dimension
    mu = mu.T  # Transpose from [100, 9] to [9, 100]
    sigma = sigma.T  # Transpose similarly

    # Expand to match batch size
    mu = mu.unsqueeze(0).expand(x.shape[0], -1, -1)  # Shape: [70, 9, 100]
    sigma = sigma.unsqueeze(0).expand(x.shape[0], -1, -1)  # Shape: [70, 9, 100]

    #print(f"After expansion -> mu shape: {mu.shape}, sigma shape: {sigma.shape}")  # Debugging

    return torch.exp(-torch.sum((x - mu) ** 2 / (2 * sigma ** 2), dim=-2))




class FuzzyInferenceBlock(nn.Module):
    def __init__(self, output_dim, i_fmap, mu, sigma):
        super(FuzzyInferenceBlock, self).__init__()
        self.output_dim = output_dim
        self.index = i_fmap
        self.mu = mu
        self.sigma = sigma
        self.mu_map = torch.transpose(out_fRules, 0, 1).to(device) * self.mu
        self.sigma_map = torch.ones((self.output_dim, n_feature), dtype=torch.float32).to(device) * self.sigma



    def forward(self, inputs):
        fMap = inputs[:, n_feature * self.index : n_feature * (self.index + 1)]
        aligned_x = fMap.unsqueeze(-1).expand(-1, -1, self.output_dim)
        phi = gaussian(aligned_x, self.mu_map, self.sigma_map)
        return phi


In [30]:
def load_data(image_dir):
    transform = transforms.Compose([
        transforms.Resize((rows, cols)),
        transforms.ToTensor()
    ])
    dataset = datasets.ImageFolder(image_dir, transform=transform)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

In [31]:
class FCNN(nn.Module):
    def __init__(self, n_femap=4, stride=2, mu=3.0, sigma=1.2, dropout=True):
        super(FCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=0)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=0)
        self.conv4 = nn.Conv2d(64, n_femap, kernel_size=4, padding=2, stride=stride)
        self.dropout = nn.Dropout(0.2) if dropout else nn.Identity()
        self.flatten = nn.Flatten()
        self.fuzzy_inference = nn.ModuleList([FuzzyInferenceBlock(n_neurons, i, mu, sigma) for i in range(n_femap)])
        self.fc = nn.Linear(n_femap * n_neurons, num_classes)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = F.relu(self.conv4(x))
        x = self.dropout(x)
        x = self.flatten(x)
        fuzzy_outputs = [block(x) for block in self.fuzzy_inference]
        x = torch.cat(fuzzy_outputs, dim=1)
        x = self.fc(x)
        return x


In [32]:
def train_model(model, train_loader, val_loader, epochs=10):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    
    for epoch in range(epochs):
        model.train()
        running_loss, correct_train, total_train = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        train_losses.append(running_loss / len(train_loader))
        train_accs.append(100 * correct_train / total_train)
        
        # Validation
        model.eval()
        val_loss, correct_val, total_val = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
        val_losses.append(val_loss / len(val_loader))
        val_accs.append(100 * correct_val / total_val)
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Train Acc: {train_accs[-1]:.2f}%, Val Acc: {val_accs[-1]:.2f}%")
    
    # Plot training progress
    fig, axs = plt.subplots(2, 1, figsize=(8, 10))
    axs[0].plot(range(1, epochs+1), train_losses, label='Train Loss')
    axs[0].plot(range(1, epochs+1), val_losses, label='Val Loss')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Loss')
    axs[0].legend()
    axs[0].set_title('Loss Progress')
    
    axs[1].plot(range(1, epochs+1), train_accs, label='Train Accuracy')
    axs[1].plot(range(1, epochs+1), val_accs, label='Val Accuracy')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Accuracy (%)')
    axs[1].legend()
    axs[1].set_title('Accuracy Progress')
    
    plt.show()

In [33]:
# Import necessary modules
import torch

# Define dataset path
image_dir = "train_images"

# Load dataset
train_loader, val_loader = load_data(image_dir)




In [34]:
# Initialize model
model = FCNN()

# Train model
trained_model = train_model(model, train_loader, val_loader, epochs=10)



torch.save(trained_model.state_dict(), "fcnn_model.pth")

KeyboardInterrupt: 